# Introduction to Animations

This note book was modified from the Beginners guide on the Digital Earth Australia [GitHub Notebook repository](https://github.com/GeoscienceAustralia/dea-notebooks)

* **Products used:** 
Landsat 8 surface reflectance (usgs_ls8c_level2_2)
* **Prerequisites:** Users of this notebook should have a basic understanding of:
    * How to run a Jupyter notebook
    * Inspecting available DEA products and measurements
    * How to load data from DEA
    * Introduction to plotting == notebooks/examples/Beginners_guide/04_Plotting.ipynb

## Background
One way to get a view of your area is to get a time based animation either in the gif format or .mp4.

the annimated_timeseries fuction from deafrica_plotting.py does this

### Additional Notes from deafrica_plotting

> Animations can be exported as .mp4 (ideal for Twitter/social media)
    and .gif (ideal for all purposes, but can have large file sizes) 
    format files, and customised to include titles and date annotations 
    or use specific combinations of input bands.



## Description
This introductory notebook demonstrates how to visualise Digital Earth Africa satellite data returned from running a datacube query.
The notebook demonstrates commonly used `xarray` animation methods, as well as custom functions provided in the [deafrica_plotting](../Scripts/dea_plotting.py) script.

Topics covered in this notebook include:

* View your area of interest prior to querying the datacube
* Querying the datacube and loading data
* Animating The Time Series
    
***

### Load packages
First we run `%matplotlib inline`, which ensures figures plot correctly in the Jupyter notebook.
We then load the `datacube` package which will allow us to load some data, and a selection of custom DE Africa functions from the `deafrica_plotting` script which is located inside the [`deafrica-sandbox-notebooks` repository](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks) in the Scripts folder.

In order to use the custom fuctions on the sandbox a copy of the python file where the fuction is defined needs to be saved in the folders on the sanbox and we need to add that folder location to the python path. In this notebook we use `sys.path.append()` to add the `Scripts` folder, which where the `deafrica_plotting` python file is allready saved, to the python path. Custom fuctions `display_map()` and `rgb()` and `animated_timeseries()` are then imported form the `deafrica_plotting` python file.

In [1]:
! pwd
! ls /home/jovyan/examples/Scripts

/notebooks/opt/notebook/03-SANDBOX-specific-demos/00-GA-africa-admin-sandbox
ls: cannot access '/home/jovyan/examples/Scripts': No such file or directory


In [2]:
%matplotlib inline

import datacube
import sys
from odc.ui import with_ui_cbk

sys.path.append("/home/jovyan/examples/Scripts")
from deafrica_plotting import display_map
from deafrica_plotting import rgb
from deafrica_plotting import animated_timeseries



ModuleNotFoundError: No module named 'deafrica_plotting'

### Connect to the datacube
We then connect to the datacube database so we can load DE Africa data. 

In [ ]:
dc = datacube.Datacube(app="05_Animating")

### Analysis parameters

The following variables are required to establish a query for this notebook:
- `lat_range`: The latitude range to analyse (e.g. `(11.72, 11.52)`). For reasonable load times, keep this to a range of ~0.1 degrees or less.
- `lon_range`: The longitude range to analyse (e.g. `(-15.63, -15.43)`). For reasonable load times, keep this to a range of ~0.1 degrees or less.
- `time_range`: The date range to analyse (e.g. `("2018-01-01", "2018-03-30")`).


In [ ]:
lat_range = (11.72, 11.52)
lon_range = (-15.63, -15.43)
time_range = ("2017-01-01", "2017-12-31")


## View the queried location
Before running a query and extracting and analysing data, it is useful to double-check that your location is correct.
The `display_map()` function shows your selected area as a red rectangle on an interactive map.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(x=lon_range, y=lat_range)

## Query and view data
The variables determined above are used here to query the DE Africa datacube using the `dc.load()` function and load data introduced in the [Loading data notebook](04_Loading_data.ipynb).
This introduction will to use Landsat 8 Surface reflectance `usgs_ls8c_level2_2`.

In [ ]:
ds = dc.load(product="ls8_usgs_sr_scene",
             measurements=['blue','green','red','nir','swir1','swir2'],
             x=lon_range,
             y=lat_range,
             time=time_range,
             output_crs='EPSG:6933',
             progress_cbk=with_ui_cbk(),
             resolution=(-30, 30))

print(ds)

## Plotting single band images

The `xarray` package provides built-in methods for plotting individual data variables or measurements.
For example, we might want to make a plot for a single measurement like the `swir1` satellite band in the data we loaded above.

To do this, we first need to access the band we are after as an `xarray.DataArray` (to revise the difference between `xarray.Dataset` and `xarray.DataArray` objects, refer back to the [Loading data](04_Loading_data.ipynb) notebook):

In [ ]:
print(ds.swir1)

### Selecting and plotting a single timestep

You can see in the object header that this `xarray.DataArray` has data for five timesteps (i.e. `<xarray.DataArray 'swir1' (time: 6, y: 834, x: 644)>`).
To make a plot for a single timestep only, we need to select it using one of the following options:

1. `.isel()`: This stands for "index selection", and lets us easily select individual timesteps from a dataset by providing the number of the observation we want. 
Counting in Python begins at 0, so to select the first timestep in the `xarray.DataArray` we can specify `.isel(time=0)`:

In [ ]:
first_timestep = ds.swir1.isel(time=0)

print(first_timestep)

2. `.sel()`: This allows us to select data using real-world coordinate labels like `time`. 
    For example, from the *Coordinates* section, we can select the first timestep (i.e. The observation for january 4th 2016) from the `xarray.DataArray` by specifying `.sel(time='2016-01-04')`:

In [ ]:
first_timestep = ds.swir1.sel(time='2017-01-06')

print(first_timestep)

We can now use the `.plot()` method to plot `swir1` data for our selected timestep:

In [ ]:
first_timestep.plot()

### Plotting multiple timesteps
It is often useful to produce plots for a single measurement across time, for example to compare change between satellite observations or summary datasets.
To plot multiple images, we can skip the `isel()` step above and plot the entire `xarray.DataArray` directly. 

To plot multiple timesteps in one figure, we need to tell the `.plot()` function to put each timestep in a different column.
We can do this by specifying `.plot(col="time")`:

In [ ]:
ds.swir1.plot(col="time")

> This kind of plotting is called "facetted plotting". For more information, refer to the [xarray documentation](http://xarray.pydata.org/en/stable/plotting.html#faceting)

### Customising plot appearance
You may notice that the plots above are dark and difficult to see clearly.
To improve the appearance of `xarray` plots, you can use the `robust=True` argument to optimise the plot colours by clipping extreme values or outliers. 
This will use the 2nd and 98th percentiles of the data to compute the color limits:

In [ ]:
ds.swir1.plot(col="time", robust=True)

We can also easily use custom colour maps/styles to visualise our data using the `cmap` parameter.

When choosing a colour map for a plot, it is important to choose a set of colours that are perceived logically by the human eye. 
The best colour maps are "perceptually uniform": these colour maps increase logically from dark to light colours, where equal increases in lightness/darkness correspond to equal changes in data values. 
Some best-practice perceptually uniform colour maps include:

```
"viridis", "plasma", "inferno", "magma", "cividis"
```

> For further reading about perceptually uniform colour maps in data visualisation, refer to the [matplotlib documentation](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html)

It is also important to consider colour blindness when selecting a colour map. 
`xarray` supports many colour maps from the "colorbrewer" family of colour maps which are optimised for colour blindness.
You can use the interactive [online tool](http://colorbrewer2.org) to browse all available colour maps, or choose from one of the following commonly used options:

```
"Greys", "Purples", "Blues", "Greens", "Oranges", "Reds",
"YlOrBr", "YlOrRd", "OrRd", "PuRd", "RdPu", "BuPu", 
"GnBu", "PuBu", "YlGnBu", "PuBuGn", "BuGn", "YlGn"
```

For a full list of available colour maps you can refer to [this list](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html).

For example, to plot our data with the perceptually uniform `magma` colour map:


In [ ]:
ds.swir1.plot(col="time", robust=True, cmap="magma")

## Plotting true or false colour RGB images

Although `xarray` makes it easy to plot single band images, plotting a three band colour photo-like image is less straightforward.

To make this easier, the `deafrica-notebooks` repository provides a custom `rgb()` function that is designed for plotting three band images.
The `rgb()` function maps three data variables/measurements from the loaded dataset to the red, green and blue channels that are used to make a three-colour image.

Providing the `red`, `green` and `blue` measurements from a dataset will produce a true colour image (akin to how humans view the landscape).
Providing `nir`, `red` and `green` measurements or any other set of three satellite bands from a dataset will produce a false colour image.

> [Learn more about colour rendering](https://en.wikipedia.org/wiki/False_color#True_color).

Hence, the `rgb()` function can be used to visualise the data returned by a query.
It requires the minimum input of:

* `ds:` The `xarray.Dataset` object
* `bands:` Three bands for display (these must be measurements found in the dataset) 
* `index:` The timestep to view, default is `0`

### Plotting a single timestep

The time dimension of your `xarray.Dataset` describes how many timesteps exist for your location during your nominated time period.
In the `rgb()` function, the `index` variable is asking for which timesteps you want to view (similar to the `isel()` example above).
Remember: counting in Python begins at 0 so to view the earliest timesteps set `index=0`:

In [ ]:
# View a red, green, blue (true colour) image of the first timestep
rgb(ds, bands=["red", "green", "blue"], index=0)

By changing the input bands, we can plot a false colour image which can provide different insights in a landscape.
This band combination (`swir1`, `nir`, `green`) emphasises growing vegetation in green, and water in deep blue:

In [ ]:
# View a swir1, nir, green (false colour) image of the first timestep
rgb(ds, bands=['swir1', 'nir', 'green'], index=0)

### Plotting multiple timesteps
As discussed in the [single band example above](#Plotting-multiple-timesteps), it can be useful to visualise multiple timesteps in a single plot (e.g. to compare change over time).

The `rgb()` function allows you to do this by providing a list of multiple images to plot using `index=[X, X, ...]`.
For example, we can plot the first and fifth image in our dataset using `index=[0, 4]` (remembering that counting in Python starts at 0):

In [ ]:
# View a true colour image for the first and fith timesteps
rgb(ds, bands=['red', 'green', 'blue'], index=[0, 4])

It is also possible to use `rgb()` to plot all timesteps in a dataset using the `col="time"` syntax we demonstrate in the [single band example above](#Plotting-multiple-timesteps): 

### Customising plot appearance
By default, `rgb()` generates plots with `robust=True` to improve the appearance of the images by clipping out the darkest and brightest 2% of pixels, using the 2nd and 98th percentiles of the data to compute the color limits

If this default provides poor results, the plot's colour stretch can be customised using the `percentile_stretch` parameter. 
This allows you to clip the most extreme minimum and maximum values in the dataset, to improve the contrast and appearance of the plot.

For example, specifying `percentile_stretch=[0.05, 0.95]` will clip out the darkest and brightest 5% of pixels, focusing the colour stretch on the remaining 90% of less extreme values:

In [ ]:
rgb(ds, 
    bands=['red', 'green', 'blue'], 
    index=0, 
    percentile_stretch=[0.05, 0.95])


# Now Animate
- description pending

In [ ]:
animated_timeseries(ds,output_path='./bolama00.gif',bands = ['swir1', 'nir', 'green'], time_dim='time',  
      percentile_stretch=(.2,.85), interval=1200)

In [ ]:
animated_timeseries(ds,output_path='./bolama10.gif',bands = ['swir1', 'nir', 'green'], time_dim='time',  
      interval=500)

In [ ]:
animated_timeseries(ds,output_path='./bolamargb.gif',bands = ['red', 'green', 'blue'], time_dim='time',  
      percentile_stretch=(.3,.75), interval=700)

In [ ]:
from notebookLib.nb_animate import nb_animated_timeseries

nb_animated_timeseries(ds,output_path='./bolama_stretch00.gif',bands = ['red', 'green', 'blue'], time_dim='time',  
      percentile_stretch=(.2,.85), interval=1200)

In [ ]:
nb_animated_timeseries(ds,output_path='./bolama_stretch10.gif',bands = ['red', 'green', 'blue'], time_dim='time',  
      percentile_stretch=(.2,.85), interval=600)

In [ ]:
help(animated_timeseries)

## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. Jupyter Notebooks
2. Products and Measurements
3. Loading data
4. **Plotting (this notebook)**


For advanced plotting and visualisation options, see:

* The [deafrica plotting](../Scripts/deafrica_plotting.py) Python script containing functions used for plotting and visualising DE Africa data.

Now that you have completed the first five tutorials, join advanced users in exploring:

* Continue exploring with some "Case studies" which provide more complex workflows and analysis examples.


***
## Additional information
**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** November 2019
